In [1]:
import sys
sys.path.append('/home/sn/ERAv1/main_repo')

In [2]:
import torch, torchvision
from torchvision import transforms
import numpy as np
import gradio as gr
from PIL import Image
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
import random

# from models.resnet import ResNet18
from models.s10_model import s10Model

import gradio as gr

/home/sn/anaconda3/envs/fastai2022/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# model = s10Model()

# new_model = model.load_from_checkpoint('s10Model.ckpt')

In [ ]:
# torch.save(new_model.state_dict(), 's10Model.pth')

In [3]:
# # model = ResNet18()
model = s10Model()

# # new_model = model.load_from_checkpoint('s10Model.ckpt')
model.load_state_dict(torch.load("s10Model.pth", map_location=torch.device('cpu')), strict=False)

<All keys matched successfully>

In [5]:

# # model = ResNet18()
# model = s10Model()

# new_model = model.load_from_checkpoint('s10Model.ckpt')
# # model.load_state_dict(torch.load("s10Model.pth", map_location=torch.device('cpu')), strict=False)

inv_normalize = transforms.Normalize(
mean=[-0.50/0.23, -0.50/0.23, -0.50/0.23],
std=[1/0.23, 1/0.23, 1/0.23]
)

new_model = model

classes = ('plane', 'car', 'bird', 'cat', 'deer',
'dog', 'frog', 'horse', 'ship', 'truck')

image_fname_prefix=['cat', 'dog', 'bird1', 'bird2', 'bird3','cars1', 'cars2', 'horse1', 'horse2', 'boat1']

target = ['cat', 'dog', 'bird', 'bird', 'bird','car', 'car', 'horse', 'horse', 'ship']

def display_gradcam(num_images=1, top_classes=3, transparency=0.5,target_layer_number=-1, max_images=5):
    
    #Set up gradcam instance
    target_layers = [model.resblock2[target_layer_number]]
    cam = GradCAM(model=model, target_layers=target_layers, use_cuda=False)
    transform = transforms.ToTensor()
    
    confidences_list = []
    visualization_list = []
    
    for i in range(num_images):
#         a = random.randint(0,max_images-1)
        f_prefix = image_fname_prefix[i]
#         print(f'example_imgs/{f_prefix}.jpeg')
        input_image = Image.open(f'example_imgs/{f_prefix}.jpeg').resize((32,32))
        org_img = np.array(input_image).astype(np.uint8)
        input_img = transform(input_image)
        input_img = input_img
        input_img = input_img.unsqueeze(0)
        print(input_img.shape)

        outputs = model(input_img)
        softmax = torch.nn.Softmax(dim=0)

        o = softmax(outputs.flatten())
    #     o = outputs.flatten()

        confidences = {classes[i]: float(o[i]) for i in range(10)}

        _, prediction = torch.max(outputs, 1)

        grayscale_cam = cam(input_tensor=input_img, targets=None)
        grayscale_cam = grayscale_cam[0, :]

        img = input_img.squeeze(0)
        img = inv_normalize(img)
        rgb_img = np.transpose(img, (1, 2, 0))
        rgb_img = rgb_img.numpy()
        visualization = show_cam_on_image(org_img/255, grayscale_cam, use_rgb=True, image_weight=transparency)
        confidences_list.append(confidences)
        visualization_list.append((visualization, classes[prediction]))
#         caption_list.append(f_prefix)

    return visualization_list, confidences_list
        
def display_misClassified(num_images=1, max_images=10):

    transform = transforms.ToTensor()
    
    confidences_list = []
    visualization_list = []
    
    for i in range(max_images):
        f_prefix = image_fname_prefix[i]
#         print(f'example_imgs/{f_prefix}.jpeg')
        input_image = Image.open(f'example_imgs/{f_prefix}.jpeg').resize((32,32))
        org_img = np.array(input_image).astype(np.uint8)
        input_img = transform(input_image)
        input_img = input_img
        input_img = input_img.unsqueeze(0)
#         print(input_img.shape)

        outputs = model(input_img)
        softmax = torch.nn.Softmax(dim=0)

        o = softmax(outputs.flatten())
    #     o = outputs.flatten()

        confidences = {classes[i]: float(o[i]) for i in range(10)}

        _, prediction = torch.max(outputs, 1)
        
        if classes[prediction] != target[i]:
            confidences_list.append(confidences)
            visualization_list.append((org_img/255, f'{classes[prediction]} / {target[i]}'))

    return visualization_list, confidences_list

def inference(input_img, img_type='GradCAM', num_images = 1, top_classes=3,transparency = 0.5, 
              target_layer_number = -1):

    transform = transforms.ToTensor()
    org_img = input_img
    print(org_img.dtype)
    input_img = transform(input_img)
    
    if img_type == 'GradCAM':
        v_list, c_list = display_gradcam(num_images, top_classes, transparency,target_layer_number, max_images=10)
    elif img_type == 'Mis-Classified':
        v_list, c_list = display_misClassified(num_images)
    else:
        target_layers = [model.resblock2[target_layer_number]]
        cam = GradCAM(model=model, target_layers=target_layers, use_cuda=False)
        input_img = input_img
        input_img = input_img.unsqueeze(0)

        outputs = new_model(input_img)
        softmax = torch.nn.Softmax(dim=0)

        o = softmax(outputs.flatten())

        confidences = {classes[i]: float(o[i]) for i in range(10)}

        _, prediction = torch.max(outputs, 1)

        grayscale_cam = cam(input_tensor=input_img, targets=None)
        grayscale_cam = grayscale_cam[0, :]

        img = input_img.squeeze(0)
        img = inv_normalize(img)
        rgb_img = np.transpose(img, (1, 2, 0))
        rgb_img = rgb_img.numpy()
        visualization = show_cam_on_image(org_img/255, grayscale_cam, use_rgb=True, image_weight=transparency)
        v_list = [(visualization, classes[prediction])]
    return  v_list #c_list

#     return top_classes, confidences_list, visualization_list

title = "CIFAR10 trained on Custom Model with GradCAM"
description = "A simple Gradio interface to infer on Custom model, and get GradCAM or Mis-Classified Images"
examples = [["example_imgs/cat.jpeg",'GradCAM',5,5, 0.5, -1], 
            ["example_imgs/dog.jpeg",'GradCAM',5,5, 0.5, -1],
            ["example_imgs/bird1.jpeg",'GradCAM',5,5, 0.5, -1],
            ["example_imgs/bird2.jpeg",'GradCAM',5,5, 0.5, -1],
            ["example_imgs/bird3.jpeg",'GradCAM',5,5, 0.5, -1],
            ["example_imgs/cars1.jpeg",'GradCAM',5,5, 0.5, -1],
            ["example_imgs/cars2.jpeg",'GradCAM',5,5, 0.5, -1],
            ["example_imgs/horse1.jpeg",'GradCAM',5,5, 0.5, -1],
            ["example_imgs/horse2.jpeg",'GradCAM',5,5, 0.5, -1],
            ["example_imgs/boat1.jpeg",'GradCAM',5,5, 0.5, -1]
           ]

demo = gr.Interface(
    inference, 
    inputs = [gr.Image(shape=(32, 32), label="Input Image"), 
              gr.Radio(['GradCAM', 'Mis-Classified'],label="Type of Images", info="Do you want to see GradCam or Mis-Classified images?"), 
              gr.Slider(1, 10, value=1,step=1, label="How Many?", info="Choose between 1 and 5"),
              gr.Slider(1, 10, value=1,step=1, label="How Many top-classes?", info="Choose between 1 and 10"),
              gr.Slider(0, 1, value = 0.5, label="Opacity of GradCAM"), 
              gr.Slider(-2, -1, value = -2, step=1, label="Which Layer for GradCAM?")], 
    outputs = [gr.Gallery(
            label="Generated images", show_label=False, elem_id="gallery"
        ).style(columns=[3], rows=[2], object_fit="contain", height="auto")
              ],
    title = title,
    description = description,
    examples = examples,
)
demo.launch()

/tmp/ipykernel_15110/2082156291.py:161: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  outputs = [gr.Gallery(


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


uint8
uint8
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
uint8


In [ ]:
# This demo needs to be run from the repo folder.
# python demo/fake_gan/run.py
import random

import gradio as gr


def fake_gan():
    images = [
        (random.choice(
            [
                "https://images.unsplash.com/photo-1507003211169-0a1dd7228f2d?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=387&q=80",
                "https://images.unsplash.com/photo-1554151228-14d9def656e4?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=386&q=80",
                "https://images.unsplash.com/photo-1542909168-82c3e7fdca5c?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxzZWFyY2h8MXx8aHVtYW4lMjBmYWNlfGVufDB8fDB8fA%3D%3D&w=1000&q=80",
                "https://images.unsplash.com/photo-1546456073-92b9f0a8d413?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=387&q=80",
                "https://images.unsplash.com/photo-1601412436009-d964bd02edbc?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=464&q=80",
            ]
        ), f"label {i}" if i != 0 else "label" * 50)
        for i in range(3)
    ]
    return images


with gr.Blocks() as demo:
    with gr.Column(variant="panel"):
        with gr.Row(variant="compact"):
            text = gr.Textbox(
                label="Enter your prompt",
                show_label=False,
                max_lines=1,
                placeholder="Enter your prompt",
            ).style(
                container=False,
            )
            btn = gr.Button("Generate image").style(full_width=False)

        gallery = gr.Gallery(
            label="Generated images", show_label=False, elem_id="gallery"
        ).style(columns=[2], rows=[2], object_fit="contain", height="auto")

    btn.click(fake_gan, None, gallery)

if __name__ == "__main__":
    demo.launch()